In [ ]:
import pandas as pd


In [ ]:
data = pd.read_csv("data train and test hashtag.csv")
data.head(2)

,Unnamed: 0,Title,Description,Topic,Descr
0,0,The Devil's Disciple,Set in Colonial America during the Revolutiona...,Fiction,set colonial america revolutionary era play te...
1,1,Taking Chances,Spice-o-meter Rating: This fun romance is a so...,Romance,spiceometer rating fun romance solid maybe sca...


In [ ]:
data = data.drop(["Unnamed: 0"], axis = 1)
data = data.dropna()
data.isna().sum()

Title          0
Description    0
Topic          0
Descr          0
dtype: int64

In [ ]:
from collections import Counter

topic_counts = Counter(data['Topic'])
sorted_topics = sorted(topic_counts, key=topic_counts.get, reverse=True)
label_mapping = {topic: i for i, topic in enumerate(sorted_topics)}
data['Label'] = data['Topic'].map(label_mapping)


In [ ]:
data

,Title,Description,Topic,Descr,Label
0,The Devil's Disciple,Set in Colonial America during the Revolutiona...,Fiction,set colonial america revolutionary era play te...,0
1,Taking Chances,Spice-o-meter Rating: This fun romance is a so...,Romance,spiceometer rating fun romance solid maybe sca...,10
2,Verdi : The Story of the Little Boy who Loved ...,This time Tapper moves his focus to Italy in h...,Biography & Autobiography,time tapper moves focus italy voyage among gre...,3
3,Success,The contents of this volume originally appeare...,Business,contents volume originally appeared weekly art...,20
4,Triplanetary,"ONE MAN DISCOVERED THE TRUTH—The Fall of Rome,...",Fiction,one man discovered truththe fall rome wars rac...,0
...,...,...,...,...,...
18095,Journey to the East,This book tells the tale of a man who goes on ...,other_1,book tells tale man goes wonderful amazing jou...,1
18096,The Monk Who Sold His Ferrari: A Fable About F...,"Wisdom to Create a Life of Passion, Purpose, a...",Biography & Autobiography,wisdom create life passion purpose peace inspi...,3
18097,I Am that,This collection of the timeless teachings of o...,Philosophy,collection timeless teachings one greatest sag...,9
18098,The Berlin Phenomenology,Since the three volume edition ofHegel's Philo...,History,since three volume edition ofhegels philosophy...,6


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


# CHiến lược train nhiều model cho từng mã liên tục

In [ ]:
models = []

In [ ]:

# Xử lý dữ liệu
X = data["Descr"]  # Mô tả làm đặc trưng đầu vào
y = data["Label"]  # Nhãn là nhãn đầu ra

# Chuyển đổi nhãn thành dạng nhãn nhị phân
mlb = MultiLabelBinarizer(classes=np.arange(31))
y_binary = mlb.fit_transform([[label] for label in y])

# Chia tập dữ liệu thành tập huấn luyện và tập kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y_binary, test_size=0.2, random_state=42)

# Tokenize và chuyển đổi văn bản thành sequences
max_words = 1000  # Số lượng từ tối đa trong từ điển
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

# Padding sequences để có cùng độ dài
max_len = 100  # Độ dài tối đa của mỗi mô tả
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_len, padding='post')

# Xây dựng và huấn luyện mô hình MLP cho mỗi nhãn (giống như trong ví dụ trước)
# ...


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:895: UserWarning: unknown class(es) [31] will be ignored
  warnings.warn(


In [ ]:
X_train_pad[0]

array([ 54,   6,  25, 826, 718, 571,  64, 170, 115, 871, 250,  25,  12,
       100,  95,  78,  36, 365,  53, 794, 172,  70, 663,  53, 237, 278,
       978,  74, 624,  18, 453, 306,  28,  66,  62, 872,  58, 571, 795,
       645,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0], dtype=int32)

In [ ]:
y_train[:,0]

array([0, 1, 0, ..., 1, 1, 0])

In [ ]:
from sklearn.utils import resample

# Số lượng mẫu bạn muốn resample cho mỗi nhãn
n_samples_resampled = 1000

models = []
for i in range(31):
    model = tf.keras.models.Sequential([
        tf.keras.layers.Embedding(max_words, 64, input_length=max_len),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    X_train_label_i = X_train_pad[y_train[:, i] == 1]
    y_train_label_i = y_train[y_train[:, i] == 1][:, i]
    X_train_non_label_i = X_train_pad[y_train[:, i] == 0]

    # Resample nhãn ít mẫu
    X_train_label_i_resampled, y_train_label_i_resampled = resample(X_train_label_i, y_train_label_i,
                                                                    n_samples=n_samples_resampled,
                                                                    random_state=42)

    # Kết hợp với mẫu không phải nhãn i
    X_train_balanced = np.concatenate((X_train_label_i_resampled, X_train_non_label_i[:n_samples_resampled]))
    y_train_label_i_resampled = np.ones(n_samples_resampled)  # Đặt nhãn của mẫu nhãn i thành 1
    y_train_non_label_i_resampled = np.zeros(n_samples_resampled)  # Đặt nhãn của mẫu không phải nhãn i thành 0
    y_train_balanced = np.concatenate((y_train_label_i_resampled, y_train_non_label_i_resampled))

    # Shuffle dữ liệu huấn luyện
    shuffle_indices = np.random.permutation(len(X_train_balanced))
    X_train_balanced = X_train_balanced[shuffle_indices]
    y_train_balanced = y_train_balanced[shuffle_indices]

    # Huấn luyện mô hình
    model.fit(X_train_balanced, y_train_balanced, epochs=10, batch_size=32, verbose=0)
    models.append(model)

# Dự đoán trên tập kiểm tra
y_pred = np.array([model.predict(X_test_pad) for model in models]).squeeze().T
y_pred_binary = np.round(y_pred)

# Đánh giá hiệu suất của mô hình
print(classification_report(y_test, y_pred_binary, target_names=[str(i) for i in range(31)]))


114/114 [==============================] - 1s 6ms/step
              precision    recall  f1-score   support

           0       0.49      0.66      0.56      1238
           1       0.17      0.48      0.25       281
           2       0.06      0.26      0.10       238
           3       0.11      0.55      0.18       179
           4       0.12      0.52      0.20       183
           5       0.46      0.82      0.59       162
           6       0.14      0.54      0.23       142
           7       0.12      0.52      0.20       103
           8       0.29      0.49      0.37       108
           9       0.12      0.58      0.20        95
          10       0.24      0.54      0.33        89
          11       0.08      0.53      0.14        58
          12       0.13      0.55      0.22        62
          13       0.10      0.48      0.16        62
          14       0.12      0.53      0.20        57
          15       0.05      0.43      0.10        49
          16       0.12   

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
from sklearn.metrics import precision_score, recall_score

# Tính toán precision và recall
precision_micro = precision_score(y_test, y_pred_binary, average='micro')
recall_micro = recall_score(y_test, y_pred_binary, average='micro')

# Tính toán accuracy từ precision và recall
accuracy = (precision_micro + recall_micro) / 2

print("Accuracy:", accuracy)


Accuracy: 0.3774985040762776
